In [5]:
import numpy as np
from itertools import combinations_with_replacement
from scipy.optimize import linear_sum_assignment

def best_partition_assignment(array1, array2):
    n = len(array1)
    k = len(array2)
    array1 = np.array(array1)
    array2 = np.array(array2)
    best_loss = np.inf
    best_indicator = None

    # All possible (k-1) split points from positions 0..n (allow empty groups)
    for split_points in combinations_with_replacement(range(n+1), k-1):
        indices = [0] + list(split_points) + [n]
        # Indices must be non-decreasing
        if any(indices[i] > indices[i+1] for i in range(len(indices)-1)):
            continue
        chunks = [array1[indices[i]:indices[i+1]] for i in range(k)]
        chunk_sums = np.array([chunk.sum() for chunk in chunks])

        # Cost matrix: |sum(chunk) - target|
        cost_matrix = np.abs(chunk_sums[:, None] - array2[None, :])
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        total_loss = cost_matrix[row_ind, col_ind].sum()

        if total_loss < best_loss:
            indicator = np.zeros((n, k))
            for i_chunk, j_target in zip(row_ind, col_ind):
                for idx in range(indices[i_chunk], indices[i_chunk + 1]):
                    indicator[idx, j_target] = 1
            best_loss = total_loss
            best_indicator = indicator

    return best_indicator, best_loss

# Example usage
array1 = [0.3, 0.5, 0.2, 0.7, 0.4, 0.8, 0.1]
array2 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.9, 1.0, 1.2]

indication, loss = best_partition_assignment(array1, array2)

if indication is None:
    print("No valid partition found (shouldn't happen with this fix)!")
else:
    np.set_printoptions(precision=3, suppress=True, linewidth=150)
    print("Indication Matrix (rows: array1, cols: array2):")
    print(indication)
    print("Total loss (sum of abs diff):", loss)

    # Optional: show which group each element is assigned to
    for i, row in enumerate(indication):
        assigned = np.where(row == 1)[0]
        if assigned.size > 0:
            print(f"array1[{i}] = {array1[i]} → group {assigned[0]} (array2={array2[assigned[0]]})")
        else:
            print(f"array1[{i}] = {array1[i]} → not assigned")


Indication Matrix (rows: array1, cols: array2):
[[0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]]
Total loss (sum of abs diff): 1.5999999999999996
array1[0] = 0.3 → group 6 (array2=1.0)
array1[1] = 0.5 → group 6 (array2=1.0)
array1[2] = 0.2 → group 6 (array2=1.0)
array1[3] = 0.7 → group 7 (array2=1.2)
array1[4] = 0.4 → group 7 (array2=1.2)
array1[5] = 0.8 → group 5 (array2=0.9)
array1[6] = 0.1 → group 2 (array2=0.3)


In [48]:
import numpy as np
from heapq import heappush, heappop

def chunk_sum(array, start, end):
    s_idx = int(np.floor(start))
    e_idx = int(np.floor(end))
    result = 0.0
    if s_idx == e_idx:
        result = (end - start) * array[s_idx]
    else:
        if s_idx < len(array):
            result += (1.0 - (start - s_idx)) * array[s_idx]
        for i in range(s_idx + 1, e_idx):
            if i < len(array):
                result += array[i]
        if e_idx < len(array):
            result += (end - e_idx) * array[e_idx]
    return result

def best_least_squares_sequential_partition(array1, array2, granularity=0.05, beam_width=100):
    n = len(array1)
    max_k = len(array2)
    array1 = np.array(array1)
    array2 = np.array(array2)
    splits = np.arange(0, n + granularity, granularity)

    best_loss = np.inf
    best_k = 1
    best_indicator = None
    best_mapped = None

    for k in range(1, max_k+1):
        beam = [(0.0, [0.0], [])]  # (cum_squared_resid, [split_points], [chunk_sums])
        for g in range(k-1):
            new_beam = []
            for total_sq_resid, split_points, chunk_sums in beam:
                last_split = split_points[-1]
                min_next = last_split + granularity
                max_next = n - (k - (g + 1)) * granularity + 1e-8
                for next_split in splits[(splits >= min_next) & (splits <= max_next)]:
                    csum = chunk_sum(array1, last_split, next_split)
                    resid = (csum - array2[g]) ** 2
                    heappush(new_beam, (total_sq_resid + resid, split_points + [next_split], chunk_sums + [csum]))
            # Prune beam
            beam = []
            for _ in range(min(beam_width, len(new_beam))):
                beam.append(heappop(new_beam))
        # Final chunk: [split_points[-1], n)
        for total_sq_resid, split_points, chunk_sums in beam:
            last_split = split_points[-1]
            csum = chunk_sum(array1, last_split, n)
            chunk_sums_full = chunk_sums + [csum]
            sq_resids = [(chunk_sums_full[j] - array2[j]) ** 2 for j in range(k)]
            total_loss = sum(sq_resids)
            if total_loss < best_loss:
                indicator = np.zeros((n, max_k))
                mapped_splits = split_points + [n]
                for g2 in range(k):
                    start, end = mapped_splits[g2], mapped_splits[g2+1]
                    s_idx = int(np.floor(start))
                    e_idx = int(np.floor(end))
                    if s_idx < n:
                        frac = min(1.0, max(0.0, 1.0 - (start - s_idx)))
                        indicator[s_idx, g2] += frac
                    for i in range(s_idx + 1, e_idx):
                        if i < n:
                            indicator[i, g2] += 1.0
                    if e_idx < n and end > e_idx:
                        frac = end - e_idx
                        indicator[e_idx, g2] += frac
                for i in range(n):
                    rs = indicator[i, :].sum()
                    if rs > 1 + 1e-8:
                        indicator[i, :] /= rs
                best_loss = total_loss
                best_k = k
                best_indicator = indicator
                best_mapped = chunk_sums_full
    return best_indicator, best_loss, best_k, best_mapped

# Example usage
array1 = [0.3, 0.5, 0.2, 0.7, 0.4, 0.8, 0.1]
array2 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.9, 1.0, 1.2]

indication, loss, used_k, mapped = best_least_squares_sequential_partition(array1, array2, granularity=0.05, beam_width=80)

np.set_printoptions(precision=3, suppress=True)
print("Best match uses k =", used_k, "groups")
print("Indication Matrix (rows: array1, cols: array2):")
print(indication)
print("Row sums:", indication.sum(axis=1))
print("Least squares loss:", loss)
print("Mapped sums for each group:", mapped)
print("Difference (mapped - target):", np.array(mapped) - np.array(array2[:used_k]))


Best match uses k = 7 groups
Indication Matrix (rows: array1, cols: array2):
[[0.433 0.55  0.017 0.    0.    0.    0.    0.   ]
 [0.    0.    0.55  0.45  0.    0.    0.    0.   ]
 [0.    0.    0.    0.85  0.15  0.    0.    0.   ]
 [0.    0.    0.    0.    0.65  0.35  0.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.3   0.7   0.   ]
 [0.    0.    0.    0.    0.    0.    1.    0.   ]]
Row sums: [1. 1. 1. 1. 1. 1. 1.]
Least squares loss: 0.11630000000000033
Mapped sums for each group: [np.float64(0.09000000000000001), np.float64(0.195), np.float64(0.29), np.float64(0.395), np.float64(0.4850000000000002), np.float64(0.8850000000000003), np.float64(0.6599999999999995)]
Difference (mapped - target): [-0.01  -0.005 -0.01  -0.005 -0.015 -0.015 -0.34 ]
